# Problem statement

To develop a simple chatbot that can answer basic questions about a specific topic.

reference: https://handsonai.medium.com/build-a-chat-bot-from-scratch-using-python-and-tensorflow-fd189bcfae45

# Setup environment

In [2]:
# import libraries
import nltk
from nltk.stem.lancaster import LancasterStemmer

import numpy as np
import json
import random
import pickle
import requests

import tensorflow as tf

# Load and Preprocess Data

reference: https://www.yourlibrary.ca/citizenship-test-answer-keys/

In [17]:
# load data
# define the URL of the JSON file
# url = "https://raw.githubusercontent.com/wpjerrykwok/LLM-chatBot/main/intents.json"
url = "https://raw.githubusercontent.com/wpjerrykwok/LLM-chatBot/main/intents_wpglibrary.json"

# make a GET request to the URL
response = requests.get(url)

# check if the request was successful
if response.status_code == 200:
    # parse the response as JSON
    raw_data = response.json()
    # print the raw data
    print(raw_data)
else:
    # handle the error
    print(f"Request failed with status code {response.status_code}")

{'intents': [{'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day'], 'responses': ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time'], 'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], 'context': ['']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure'], 'context': ['']}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], 'context': ['']}, {'tag': 'librarycard', 'patterns': ['why library card is important', 'why need a library card'], 'responses': ['Winnipeg Public Library has 20 locations serving the citizens of Winnipeg. The Librar

In [18]:
stemmer = LancasterStemmer()

In [19]:
try:
    with open('data.pickle', 'rb') as data_file:
        words, labels, training, output = pickle.load(data_file)
except:
# get the words and labels
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in raw_data['intents']:
        for pattern in intent['patterns']:
            tokenized_words = nltk.word_tokenize(pattern)
            words.extend(tokenized_words)
            docs_x.append(tokenized_words)
            docs_y.append(intent['tag'])

        if intent['tag'] not in labels:
            labels.append(intent['tag'])

    # stem the words
    words = [stemmer.stem(w.lower()) for w in words if w != '?']
    words = sorted(list(set(words)))
    labels = sorted(labels)

    # create training and output data
    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    # one hot encoding
    for x, doc in enumerate(docs_x):
        bag = []

        stemmed_words = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in stemmed_words:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

    # convert to numpy arrays
    training = np.array(training)
    output = np.array(output)

    # save data
    with open('data.pickle', 'wb') as data_file:
        pickle.dump((words, labels, training, output), data_file)

# Train the model

In [20]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(8, input_shape=[len(training[0])]))
model.add(tf.keras.layers.Dense(8))
model.add(tf.keras.layers.Dense(len(output[0]), activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

try:
    model.load_weights('model.keras')
except:
    model.fit(training, output, epochs=1000, batch_size=8)
    model.save('model.keras')

Epoch 1/1000
6/6 [==============================] - 0s 800us/step - loss: 2.9006 - accuracy: 0.0435
Epoch 2/1000
6/6 [==============================] - 0s 0s/step - loss: 2.8712 - accuracy: 0.0435
Epoch 3/1000
6/6 [==============================] - 0s 2ms/step - loss: 2.8461 - accuracy: 0.0652
Epoch 4/1000
6/6 [==============================] - 0s 1ms/step - loss: 2.8226 - accuracy: 0.0652
Epoch 5/1000
6/6 [==============================] - 0s 0s/step - loss: 2.7981 - accuracy: 0.0870
Epoch 6/1000
6/6 [==============================] - 0s 3ms/step - loss: 2.7768 - accuracy: 0.0870
Epoch 7/1000
6/6 [==============================] - 0s 3ms/step - loss: 2.7555 - accuracy: 0.0870
Epoch 8/1000
6/6 [==============================] - 0s 3ms/step - loss: 2.7345 - accuracy: 0.0870
Epoch 9/1000
6/6 [==============================] - 0s 3ms/step - loss: 2.7136 - accuracy: 0.0870
Epoch 10/1000
6/6 [==============================] - 0s 0s/step - loss: 2.6923 - accuracy: 0.0870
Epoch 11/1000
6/6 [=

In [21]:
#
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    tokenized_words = nltk.word_tokenize(s)
    stemmed_words = [stemmer.stem(w.lower()) for w in tokenized_words]

    for w in stemmed_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1

    return np.array(bag)

In [22]:
def chat():
    print('Start talking with the bot! (type quit to stop)')
    while True:
        inp = input('You: ')
        if inp.lower() == 'quit':
            break

        results = model.predict(np.array([bag_of_words(inp, words)]))
        results_index = np.argmax(results)
        tag = labels[results_index]

        for intent in raw_data['intents']:
            if intent['tag'] == tag:
                responses = intent['responses']

        print(random.choice(responses))

In [23]:
chat()

Start talking with the bot! (type quit to stop)
1/1 [==============================] - 0s 55ms/step
Good to see you again
1/1 [==============================] - 0s 16ms/step
See you!
1/1 [==============================] - 0s 17ms/step
15 Poseidon Bay
1/1 [==============================] - 0s 16ms/step
Library cards are free if you live or pay property or business taxes in the City of Winnipeg.
1/1 [==============================] - 0s 15ms/step
Bill and Helen Norrie; Charleswood; Millennium; Munroe; Osborne
1/1 [==============================] - 0s 17ms/step
625 Osborne St.
